In [ ]:
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Flatten, Dense, Lambda

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception # TensorFlow ONLY
from tensorflow.keras.applications import VGG16


from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers.core import Dense, Dropout, Flatten, Activation
#from tensorflow.keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array
#from tensorflow.keras.utils import np_utils
import os
import numpy as np
from tqdm import tqdm 

In [ ]:
import os
import tensorflow.keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import efficientnet.tfkeras as efn 
from tensorflow.keras.models import Model, load_model
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm


plt.rcParams['savefig.dpi'] =300
plt.rcParams['figure.dpi'] = 300

model_a = load_model('efn2.h5')

def load_image(img_url):
    image = load_img(img_url,target_size=(256,256))
    image = img_to_array(image)
    image /= 255
    image = np.expand_dims(image,axis=0)
    return image

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
import efficientnet.tfkeras as efn 

model_builder = keras.applications.xception.Xception
img_size = (256, 256)
preprocess_input = keras.applications.xception.preprocess_input
decode_predictions = keras.applications.xception.decode_predictions

last_conv_layer_name = "top_conv"

# The local path to our target image
img_path = '/mnt/B002.BMP'

#display(Image(img_path))


import matplotlib.image as imgplt
import matplotlib.pyplot as plt
#x = imgplt.imread(img_path)

x = load_img(img_path,target_size=(256,256))

plt.imshow(x)


import numpy as np
image1 = load_img(img_path,target_size=(256,256))
image1 = img_to_array(image1)
image1 /= 255

#IMAGE1 = np.asarray(image1)
image1 =image1.reshape(1,256,256,3)
Y_pred1 = model_a.predict(image1)
#Y_pred1 = model_a.predict(IMAGE1[np.newaxis,...])

print(Y_pred1.max(),np.argmax(Y_pred1),Y_pred1)

plt.axis('off')

In [ ]:
# Prepare image
img_array = preprocess_input(get_img_array(img_path, size=img_size))


# Print what the top predicted class is
preds = model_a.predict(img_array)

#print("Predicted:", decode_predictions(preds, top=1)[0])

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, model_a, last_conv_layer_name)


# Display heatmap
plt.matshow(heatmap)
plt.axis('off')
plt.show()

In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.6):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path,target_size=(256,256))
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    #display(Image(cam_path))
    cam = imgplt.imread(cam_path)
    plt.axis('off')
    plt.imshow(cam)


save_and_display_gradcam(img_path, heatmap)